In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from scipy.optimize import linprog

In [2]:
victoria_secret = pd.read_csv('Dataset/Intermediate/vs_products_intermediate.csv')
victoria_secret.head()

,product_name,product_category,description,product_category_group,mrp,price,size_popularity,color_popularity,%discount
0,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Bikini Panty,Dotted mesh in the ultimate everyday shape mak...,panties,10.500000,10.500000,0.126413,0.154872,0.00
1,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Thong Panty,Dotted mesh in a sexy little shape makes this ...,panties,10.500000,10.500000,0.126413,0.154872,0.00
2,Body by Victoria Cheekini Panty,Cheekini Panty,"The comfiest way to show a little cheek peek, ...",panties,14.500000,7.142542,0.754805,0.163567,50.74
3,Body by Victoria Daisy Lace Slip,Daisy Lace Slip,What dreams are made of: this little slip in f...,slip,52.000000,34.990000,0.126413,0.155092,32.71
4,Body by Victoria Demi Bra,Demi Bra,A lower cut makes this supersoft bra perfect f...,bras,50.617021,50.617021,1.535736,0.311891,0.00


In [3]:
brands_data = pd.read_csv('Dataset/Intermediate/all_products.csv')
brands_data.head()

,product_name,brand_name,product_category,description,product_category_wide,mrp,price,%discount
0,"""I DO"" Signature Lace Cheeky Hipster with Gift...",hanky panky,Collections,"With ""I DO"" emblazoned on this Hanky Panky lac...",panties,40.0,40.0,0.0
1,"""I DO"" Signature Lace Cheeky Hipster with Gift...",hanky panky,Panties,"With ""I DO"" emblazoned on this Hanky Panky lac...",panties,40.0,40.0,0.0
2,'After Midnight' Lace Open Gusset G-String,hanky panky,Bridal Lingerie,Stretch signature lace fashions an alluring do...,other,27.0,27.0,0.0
3,'After Midnight' Lace Open Gusset G-String,hanky panky,Sexy Lingerie,Stretch signature lace fashions an alluring do...,other,27.0,27.0,0.0
4,'After Midnight' Lace Open Gusset G-String,hanky panky,Women's Panties,Stretch signature lace fashions an alluring do...,other,27.0,27.0,0.0


In [4]:
brands_data.shape

(5982, 8)

In [5]:
brand_count_list = []
def calculate_brand_count(x,y,z):
    count = {}
    for i in x.split():
        count[i.lower()] = count.get(i.lower(), 0) + 1
    brand_count_list.append([count, y, z])
brands_data.apply(lambda x: calculate_brand_count(x['product_name'], x['mrp'], x['product_category_wide']), axis=1)
brand_count_list[0]

[{'"i': 1,
  'do"': 1,
  'signature': 1,
  'lace': 1,
  'cheeky': 1,
  'hipster': 1,
  'with': 1,
  'gift': 1,
  'box': 1},
 40.0,
 'panties']

In [6]:
def calculate_competitor_price(x, y, z):
    for i in x.split():
        price = []
        for j in brand_count_list:
            if ((j[0].get(i.lower(), 0) >= 1) & (y == j[2])):
                price.append(j[1])
        if len(price) > 0:
            return round((sum(price)/len(price)), 6)
        else:
            return z
victoria_secret['competitor_price'] = victoria_secret.apply(lambda x: calculate_competitor_price(x['product_name'], x['product_category_group'], x['mrp']), axis = 1)

In [7]:
victoria_secret.head()

,product_name,product_category,description,product_category_group,mrp,price,size_popularity,color_popularity,%discount,competitor_price
0,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Bikini Panty,Dotted mesh in the ultimate everyday shape mak...,panties,10.500000,10.500000,0.126413,0.154872,0.00,32.500000
1,Allover Lace from Cotton Lingerie NEW! Dotted ...,Dotted Mesh Thong Panty,Dotted mesh in a sexy little shape makes this ...,panties,10.500000,10.500000,0.126413,0.154872,0.00,32.500000
2,Body by Victoria Cheekini Panty,Cheekini Panty,"The comfiest way to show a little cheek peek, ...",panties,14.500000,7.142542,0.754805,0.163567,50.74,17.954545
3,Body by Victoria Daisy Lace Slip,Daisy Lace Slip,What dreams are made of: this little slip in f...,slip,52.000000,34.990000,0.126413,0.155092,32.71,52.000000
4,Body by Victoria Demi Bra,Demi Bra,A lower cut makes this supersoft bra perfect f...,bras,50.617021,50.617021,1.535736,0.311891,0.00,40.590782


In [8]:
victoria_secret.drop(['description', 'product_category', 'size_popularity', 'color_popularity'], axis = 1, inplace = True)
victoria_secret.head()

,product_name,product_category_group,mrp,price,%discount,competitor_price
0,Allover Lace from Cotton Lingerie NEW! Dotted ...,panties,10.500000,10.500000,0.00,32.500000
1,Allover Lace from Cotton Lingerie NEW! Dotted ...,panties,10.500000,10.500000,0.00,32.500000
2,Body by Victoria Cheekini Panty,panties,14.500000,7.142542,50.74,17.954545
3,Body by Victoria Daisy Lace Slip,slip,52.000000,34.990000,32.71,52.000000
4,Body by Victoria Demi Bra,bras,50.617021,50.617021,0.00,40.590782


In [9]:
n = victoria_secret.shape[0]
r = n
c = n*3
A  = []
for i in range(r):
    s = [0 for i in range(c)]
    s[3*i],s[3*i+1],s[3*i + 2] = 1,1,1
    A.append(s)

In [10]:
B = np.ones(n, dtype='int').reshape(n,-1)
# B = [1 for i in range(victoria_secret.shape[0])]

In [11]:
R = []
def calculate_array(m,p,c):
    p1 = max(m,c)
    p3 = min(m,c)
    p2 = p3 + round(abs(c-m)/2.0, 6)
    
    d1 = min(p,m)
    d3 = max(p,m)
    d2 = d1 + round(abs(p-m)/2.0, 6)
    R.extend([p1*d1, p2*d2, p3*d3])

victoria_secret.apply(lambda x: calculate_array(x['mrp'],x['price'],x['competitor_price']), axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
664    None
665    None
666    None
667    None
668    None
Length: 669, dtype: object

In [12]:
equation = np.array(R)*-1
left = [i for i in A]
right = B
bounds = (0,1)

res = linprog(equation, A_eq=left, b_eq=right, bounds = [bounds])

In [13]:
print('Optimal value:', round(res.fun*-1, ndigits=2),
      '\nx values:', res.x,
      '\nNumber of iterations performed:', res.nit,
      '\nStatus:', res.message)

Optimal value: 832750.31 
x values: [1.00000000e+00 8.98212567e-13 2.90322731e-13 ... 3.28850963e-13
 1.16183054e-12 1.00000000e+00] 
Number of iterations performed: 9 
Status: Optimization terminated successfully.


In [14]:
result = [round(i, 6) for i in res.x]
result[:15]

[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]

In [15]:
victoria_secret.reset_index(inplace = True)

In [16]:
def optimal_price(m,c,i):
    p1 = max(m,c)
    p3 = min(m,c)
    p2 = p3 + round(abs(c-m)/2.0, 6)
    return p1 if result[i] > 0 else p2 if result[i+1] > 0 else p3

victoria_secret['optimal_price'] = victoria_secret.apply(lambda x: optimal_price(x['mrp'],x['competitor_price'], x['index']), axis = 1)

In [17]:
victoria_secret.drop(['index','price','%discount'], inplace = True, axis = 1)

In [18]:
victoria_secret.head(20)

,product_name,product_category_group,mrp,competitor_price,optimal_price
0,Allover Lace from Cotton Lingerie NEW! Dotted ...,panties,10.500000,32.500000,32.500000
1,Allover Lace from Cotton Lingerie NEW! Dotted ...,panties,10.500000,32.500000,10.500000
2,Body by Victoria Cheekini Panty,panties,14.500000,17.954545,16.227272
3,Body by Victoria Daisy Lace Slip,slip,52.000000,52.000000,52.000000
4,Body by Victoria Demi Bra,bras,50.617021,40.590782,40.590782
5,Body by Victoria Demi Bra,bras,45.924802,40.590782,40.590782
6,Body by Victoria Demi Bra,bras,52.979409,40.590782,40.590782
7,Body by Victoria Flutter Bandeau,bras,34.500000,40.590782,37.545391
8,Body by Victoria Front-Close Unlined Bralette,bralettes,34.948718,34.948718,34.948718
9,Body by Victoria High-neck Halter Bralette,bralettes,34.500000,34.500000,34.500000
